# Wrapping the model as a reddit bot

After training and saving the trained model, we want to wrap it as a reddit bot that produces a score for every post made on the subreddit r/ProperTechno as a post reply. We will use the Reddit API. Might need to do this in PyCharm.

In [ ]:
import praw
import time

# create a Reddit instance
reddit = praw.Reddit(client_id='your_client_id',
                     client_secret='your_client_secret',
                     user_agent='your_user_agent',
                     username='your_username',
                     password='your_password')

# set up youtube_dl options
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192'
    }]
}

# # Load the trained seqential model
# model = keras.models.load_model('my_model.h5')
# OR
# Load the random forrest model
with open('path/to/saved/model.pkl', 'rb') as f:
    model = pickle.load(f)

# specify the subreddit to search
subreddit = reddit.subreddit('ProperTechno')

# Read posted ids from file
with open('posted_ids.txt', 'r') as f:
    posted_ids = f.read().splitlines()

# loop through the latest 20 posts in the subreddit
for post in subreddit.new(limit=20):
    if post.id in posted_ids:

        # check if the post is a link post
        if post.is_self == False:

            # fetch the url of the post
            url = post.url
            if 'youtube.com' in url:
                
                # Download track
                with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([url])
                    
                # Read audio file
                track = 0 # To ad

            # check if it's a bandcamp link
            elif 'bandcamp.com' in url:
                # initialize a BandcampDL object with the track URL
                dl = bandcamp_dl.BandcampDL(track_url)

                # get the track information
                track_info = dl.get_track_info()

                # get the direct download link of the track
                track_download_link = dl.get_download_link(track_info)

                # download the track to a file
                dl.download(track_download_link, file_name=f"{track_info['artist']} - {track_info['title']}.mp3")

                # Read audio file
                track = 0 # To add
            
            # Load track
            y, sr = librosa.load(track, res_type='kaiser_fast', mono=False)
  
            # Compute the mfcc features
            mfccs = librosa.feature.mfcc(y, sr=sr, n_mfcc=40)
            mfccs_scaled = np.mean(mfccs.T,axis=0)

            # Predict the class using the trained model
            prediction = model.predict_classes(mfccs_scaled.reshape(1,-1))

            # Make the comment on the post
            post.reply('Proper bot thinks this track is {} likely to be proper techno. Do you agree?'.format(prediction[0]))

            # Append posted id to file
            with open('posted_ids.txt', 'a') as f:
                f.write(post.id + '\n')

            else:
                print("Already posted on", post.id)

    else:
        
        print("Post already processed")
        
    # wait for a bit to avoid hitting the API too frequently
    time.sleep(1)
    